In [ ]:
from azureml.core import Workspace, Experiment, Dataset, Datastore, ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

workspace = Workspace.from_config()

In [ ]:
myenv = Environment.from_conda_specification(name="arima-env", file_path="arima-env.yml")
docker_config = DockerConfiguration(use_docker=True)

aml_compute = 'demo-cpucluster1'  # <== Compute Cluster Name
source_directory = './scripts'
#script='transactions_arima.py'
script='trainAndRegisterModel.py'

In [ ]:
src = ScriptRunConfig(source_directory=source_directory,
                      script=script,
                      compute_target=aml_compute,
                      environment=myenv,
                      docker_runtime_config=docker_config)

In [ ]:
experiment_name = "arima-mlops-remote"
experiment = Experiment(workspace = workspace, name = experiment_name)

run = experiment.submit(config=src)
run

In [ ]:
run.wait_for_completion(show_output=True, wait_post_processing=True)